In [11]:
#Import Necessary Packages
!pip install xgboost==2.1.4

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   --- ------------------------------------ 9.7/124.9 MB 54.9 MB/s eta 0:00:03
   ------ --------------------------------- 21.2/124.9 MB 55.9 MB/s eta 0:00:02
   ----------- ---------------------------- 34.6/124.9 MB 57.8 MB/s eta 0:00:02
   --------------- ------------------------ 47.7/124.9 MB 59.5 MB/s eta 0:00:02
   ------------------- -------------------- 60.8/124.9 MB 60.6 MB/s eta 0:00:02
   ----------------------- ---------------- 73.9/124.9 MB 61.2 MB/s eta 0:00:01
   --------------------------- ------------ 87.0/124.9 MB 61.7 MB/s eta 0:00:01
   ------------------------------- ------- 100.1/124.9 MB 62.0 MB/s eta 0:00:01
   ----------------------------------- --- 113.8/124.9 MB 62.1 MB/s eta 0:00:01
   --------------------------------------  124.8/124.9 MB 62.7 MB/s eta 0:00:01
   --------------------------------------- 124.9/124.9 MB 57.0 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing

In [13]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import pandas as pd

In [15]:
#read in data
df =(pd.read_csv("alzheimers_disease_data.csv"))
df

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis,DoctorInCharge
0,4751,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,...,0,0,1.725883,0,0,0,1,0,0,XXXConfid
1,4752,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,...,0,0,2.592424,0,0,0,0,1,0,XXXConfid
2,4753,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,...,0,0,7.119548,0,1,0,1,0,0,XXXConfid
3,4754,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,...,0,1,6.481226,0,0,0,0,0,0,XXXConfid
4,4755,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,...,0,0,0.014691,0,0,1,1,0,0,XXXConfid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2144,6895,61,0,0,1,39.121757,0,1.561126,4.049964,6.555306,...,0,0,4.492838,1,0,0,0,0,1,XXXConfid
2145,6896,75,0,0,2,17.857903,0,18.767261,1.360667,2.904662,...,0,1,9.204952,0,0,0,0,0,1,XXXConfid
2146,6897,77,0,0,1,15.476479,0,4.594670,9.886002,8.120025,...,0,0,5.036334,0,0,0,0,0,1,XXXConfid
2147,6898,78,1,3,1,15.299911,0,8.674505,6.354282,1.263427,...,0,0,3.785399,0,0,0,0,1,1,XXXConfid


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2149 entries, 0 to 2148
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PatientID                  2149 non-null   int64  
 1   Age                        2149 non-null   int64  
 2   Gender                     2149 non-null   int64  
 3   Ethnicity                  2149 non-null   int64  
 4   EducationLevel             2149 non-null   int64  
 5   BMI                        2149 non-null   float64
 6   Smoking                    2149 non-null   int64  
 7   AlcoholConsumption         2149 non-null   float64
 8   PhysicalActivity           2149 non-null   float64
 9   DietQuality                2149 non-null   float64
 10  SleepQuality               2149 non-null   float64
 11  FamilyHistoryAlzheimers    2149 non-null   int64  
 12  CardiovascularDisease      2149 non-null   int64  
 13  Diabetes                   2149 non-null   int64

In [19]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [25]:
#Drop Unnecessary Columns
df = df.drop(columns=['PatientID', 'DoctorInCharge'])
df

,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,SleepQuality,FamilyHistoryAlzheimers,CardiovascularDisease,Diabetes,Depression,HeadInjury,Hypertension,SystolicBP,DiastolicBP,CholesterolTotal,CholesterolLDL,CholesterolHDL,CholesterolTriglycerides,MMSE,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis
0,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,9.025679,0,0,1,1,0,0,142,72,242.366840,56.150897,33.682564,162.189143,21.463532,6.518877,0,0,1.725883,0,0,0,1,0,0
1,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,7.151293,0,0,0,0,0,0,115,64,231.162595,193.407995,79.028477,294.630909,20.613267,7.118696,0,0,2.592424,0,0,0,0,1,0
2,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,9.673574,1,0,0,0,0,0,99,116,284.181858,153.322762,69.772292,83.638324,7.356249,5.895077,0,0,7.119548,0,1,0,1,0,0
3,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,8.392554,0,0,0,0,0,0,118,115,159.582240,65.366637,68.457491,277.577358,13.991127,8.965106,0,1,6.481226,0,0,0,0,0,0
4,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,5.597238,0,0,0,0,0,0,94,117,237.602184,92.869700,56.874305,291.198780,13.517609,6.045039,0,0,0.014691,0,0,1,1,0,0
5,86,1,1,1,30.626886,0,4.140144,0.211062,1.584922,7.261953,0,0,1,0,0,0,168,62,280.712539,198.334629,79.080503,263.943655,27.517529,5.510144,0,0,9.015686,1,0,0,0,0,0
6,68,0,3,2,38.387622,1,0.646047,9.257695,5.897388,5.477686,0,0,0,0,1,0,143,88,263.734149,52.470670,66.533369,216.489175,1.964413,6.062124,0,0,9.236328,0,0,0,0,1,0
7,75,0,0,1,18.776009,0,13.723826,4.649451,8.341903,4.213210,0,0,0,0,0,0,117,63,151.383137,69.623510,77.346816,210.570866,10.139568,3.401374,0,0,4.517248,1,0,0,0,1,1
8,72,1,1,0,27.833188,0,12.167848,1.531360,6.736882,5.748224,0,0,0,0,0,1,117,119,233.605755,144.045740,43.075893,151.164186,25.820732,7.396061,0,1,0.756232,0,0,1,0,0,0
9,87,0,0,0,35.456302,1,16.028688,6.440773,8.086019,7.551773,0,1,0,0,0,0,130,78,281.630050,130.497580,74.291247,144.175974,28.388409,1.148904,0,1,4.554394,0,0,0,0,0,0


In [27]:
print(df.columns[0])

Age


In [29]:
for feature in df.columns:
    total = 0
    for x in df[feature]:
        total += x
        average = total/2149
    print(f"{feature}: {average}")

Age: 74.90879478827361
Gender: 0.5062819916240112
Ethnicity: 0.6975337366216845
EducationLevel: 1.286644951140065
BMI: 27.655696907947878
Smoking: 0.28850628199162404
AlcoholConsumption: 10.039441708849221
PhysicalActivity: 4.920202144071197
DietQuality: 4.99313791365938
SleepQuality: 7.0510810719287935
FamilyHistoryAlzheimers: 0.25221033038622614
CardiovascularDisease: 0.14425314099581202
Diabetes: 0.15076779897626802
Depression: 0.20055839925546767
HeadInjury: 0.09260120986505352
Hypertension: 0.14890646812470917
SystolicBP: 134.26477431363426
DiastolicBP: 89.84783620288506
CholesterolTotal: 225.19751887854815
CholesterolLDL: 124.33594410511417
CholesterolHDL: 59.463533141070265
CholesterolTriglycerides: 228.2814961784597
MMSE: 14.755131978258252
FunctionalAssessment: 5.080054971165188
MemoryComplaints: 0.2080037226617031
BehavioralProblems: 0.15681712424383434
ADL: 4.982958398765941
Confusion: 0.20521172638436483
Disorientation: 0.15821312238250348
PersonalityChanges: 0.150767798976

In [31]:
#Shape of Dataframe
df.shape

(2149, 33)

In [33]:
#Input features
X = df.drop(columns = ['Diagnosis'])
X.shape

(2149, 32)

In [35]:
#Target variable
y = df['Diagnosis']
y.shape

(2149,)

In [37]:
#Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [119]:
#GridSearch to tune hyperparameters - used X_train, X_test, y_train, y_test to train and evaluate the dataset - best parameters are listed below
"""
Best XGBoost Params: {'gamma': 0.5, 'learning_rate': 0.01, 'max_depth': 6, 'min_child_weight': 1,
'n_estimators': 300, 'reg_lambda': 10, 'subsample': 1.0}
XGBoost Accuracy: 0.9558139534883721

xgb_params = {
    'n_estimators': [200, 300, 400],
    'max_depth': [3, 6, 9, 12],
    'learning_rate': [0.005, 0.01, 0.05, 0.1, 0.2, 0.3],
    'subsample': [0.75, 1.0],
    'min_child_weight': [1, 3],
    'reg_lambda': [5,10],
    'gamma': [0.25, 0.5, 0.75]

}
"""

In [39]:
#Define the hyperparameters

best_alzheimers_classification_model = XGBClassifier(
    gamma=0.5,
    learning_rate=0.01,
    max_depth=6,
    min_child_weight=1,
    n_estimators=300,
    reg_lambda=10,
    subsample=1.0,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

In [41]:
#Train the model on the complete dataset
best_alzheimers_classification_model.fit(X,y)

C:\Users\adria\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:19:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=0.5, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=300,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [61]:
model_path = r"C:\Users\adria\OneDrive\Desktop\Alzheimer's_Detection_Project\ml_model\model.pkl"

In [57]:
import os
print(os.path.exists(model_path))

False


In [65]:
import joblib
joblib.dump(best_alzheimers_classification_model, "model.pkl")
#best_alzheimers_classification_model.save_model(model_path)

['model.pkl']